In [2]:
import numpy as np

In [11]:
PATH_TO_DATA = "naivebayes-21\\trg.csv"

In [29]:
list_data = []
with open(PATH_TO_DATA) as csv_file:
    for line in csv_file:
        line_lst = line.split(',')
        line_lst[-1] = line_lst[-1].replace('\n', '')

        list_data.append(line_lst)

full_csv_data = np.array(list_data)

headers = full_csv_data[0]
data = np.asarray(full_csv_data[1:])

In [32]:
print(data[:10])

[['1' 'B'
  '"the 4 202 353 bp genome of the alkaliphilic bacterium bacillus halodurans c-125 contains 4066 predicted protein coding sequences cdss 2141 527 of which have functional assignments 1182 29 of which are conserved cdss with unknown function and 743 18 3 of which have no match to any protein database among the total cdss 88 match sequences of proteins found only in bacillus subtilis and 667 are widely conserved in comparison with the proteins of various organisms including bsubtilis the b halodurans genome contains 112 transposase genes indicating that transposases have played an important evolutionary role in horizontal gene transfer and also in internal genetic rearrangement in the genome strain c-125 lacks some of the necessary genes for competence such as coms srfa and rapc supporting the fact that competence has not been demonstrated experimentally in c-125 there is no paralog of tupa encoding teichuronopeptide which contributes to alkaliphily in the c-125 genome and an 